In [5]:

import cv2
import os
# Import necessary libraries
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from zipfile import ZipFile
data="Autism_vol10.zip"
with ZipFile(data,'r') as zip:
  zip.extractall()
  print('Done')

# Define data paths
train_data_dir = 'Autism_vol8/train'
validation_data_dir = 'Autism_vol8/validation'
test_data_dir = 'Autism_vol8/test'




Done


In [6]:
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [7]:
# Set hyperparameters
batch_size = 64
epochs = 40
image_size = (256, 256)  # Adjust the image size as needed
num_classes=2
# Data augmentation for training
train_datagen = ImageDataGenerator(
        rescale=1./255,
        zoom_range=0.20,
        fill_mode="nearest")
validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Start ImageClassification Model
train_generator = train_datagen.flow_from_directory(train_data_dir,

        target_size=(256, 256),
        color_mode='rgb',                                               
        batch_size=32,
        class_mode='categorical',
        subset='training')

validation_generator = validation_datagen.flow_from_directory(
        validation_data_dir,
        color_mode='rgb',
        target_size=(256, 256),
        batch_size=32,
        class_mode='categorical',  # only data, no labels
        shuffle=False)
test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)


Found 3518 images belonging to 2 classes.
Found 726 images belonging to 2 classes.
Found 352 images belonging to 2 classes.


In [8]:
# Create CNN model
model = Sequential([
    Conv2D(128, (3, 3), activation='relu', input_shape=(image_size[0], image_size[1], 3)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((3, 3)),
    Conv2D(128, (3, 3), activation='relu'),
    Conv2D(64, (3, 3), activation='relu' ),
    MaxPooling2D((3, 3)),
    Conv2D(64, (3, 3), activation='relu',),
    MaxPooling2D((3, 3)),
    Flatten(),
    Dense(256, activation='relu'),
    Dense(128, activation='relu'),
   
    Dense(num_classes, activation='softmax')  # num_classes is the number of classes in your dataset
])

# Compile the model
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples//batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples//batch_size
)
model.summary()
# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(test_generator, steps=test_generator.samples//batch_size)

print(f"Test accuracy: {test_acc}")

Epoch 1/40
54/54 [==============================] - 1578s 29s/step - loss: 0.6272 - accuracy: 0.6834 - val_loss: 0.4849 - val_accuracy: 1.0000
Epoch 2/40
54/54 [==============================] - 1505s 28s/step - loss: 0.6208 - accuracy: 0.6848 - val_loss: 0.3344 - val_accuracy: 1.0000
Epoch 3/40
54/54 [==============================] - 1518s 28s/step - loss: 0.6030 - accuracy: 0.6802 - val_loss: 0.4488 - val_accuracy: 0.9773
Epoch 4/40
54/54 [==============================] - 1490s 28s/step - loss: 0.5703 - accuracy: 0.7390 - val_loss: 1.0154 - val_accuracy: 0.4403
Epoch 5/40
54/54 [==============================] - 1425s 26s/step - loss: 0.5466 - accuracy: 0.7462 - val_loss: 0.7398 - val_accuracy: 0.6676
Epoch 6/40
54/54 [==============================] - 1444s 27s/step - loss: 0.5621 - accuracy: 0.7373 - val_loss: 0.4895 - val_accuracy: 0.8153
Epoch 7/40
54/54 [==============================] - 1442s 27s/step - loss: 0.5217 - accuracy: 0.7740 - val_loss: 0.6757 - val_accuracy: 0.7074

In [10]:
from PIL import Image 
import PIL 
import matplotlib.pyplot as plt 
def plot(history):
    plt.figure(1) 
     # summarize history for accuracy  
 
    plt.subplot(211)  
    plt.plot(history.history['accuracy'])  
    plt.plot(history.history['val_accuracy'])  
    plt.title('accuracy vs val_accuracy')  
    plt.ylabel('accuracy')  
    plt.xlabel('epoch')  
    plt.legend(['Train', 'Validation'], loc='lower right')  
 
     # summarize history for loss  
 
    plt.subplot(212)  
    plt.plot(history.history['loss'])  
    plt.plot(history.history['val_loss'])  
    plt.title('loss vs val_loss')  
    plt.ylabel('loss')  
    plt.xlabel('epoch')  
    plt.legend(['Train', 'Validation'], loc='upper right')
    plt.tight_layout()
    plt.show()